In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "MNIST",
            # 'dataset': "FMNIST",
            'random_class_idx': True,

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 20,
            'batch_size_train': 10, # should be smaller than per_task_memory_examples 
            'batch_size_memory': 10,
            'batch_size_validation': 256,
            'tau': 10,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.001,
            'lambda': .01,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=MNIST/seed=10_epoch=1_lr=0.001_alpha=0.001_tau=10


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                    per_task_memory_examples=params['per_task_memory_examples'],
                    per_task_examples = params['per_task_examples'],
                    random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)



[8 2 5 6 3 1 0 7 4 9]


In [3]:
from algorithms.gss import GSSGreedy as Algorithm
from metrics import MetricCollector2
from backbones import MLP2Layers2

backbone = MLP2Layers2(
    input_dim=input_dim, 
    hidden_dim_1=256, 
    hidden_dim_2=256, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])
algorithm = Algorithm(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [4]:
from trainers.baselines import BaseMemoryContinualTrainer as ContinualTrainer
trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [5]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 97.22394664374512, 'loss': 0.0003083910094806228, 'fairness': 2.2627063336676025}
---------------------------- Task 2 -----------------------
[2] Eval metrics for task 1 >> {'accuracy': 65.15169603489169, 'loss': 0.006459504514486934, 'fairness': 5.0741766550467196}
[2] Eval metrics for task 2 >> {'accuracy': 97.67464448542836, 'loss': 0.00031341936237908697, 'fairness': 0.028904575114446773}
---------------------------- Task 3 -----------------------
[3] Eval metrics for task 1 >> {'accuracy': 46.11249064832943, 'loss': 0.011692871362832584, 'fairness': 3.476831733600752}
[3] Eval metrics for task 2 >> {'accuracy': 72.0052753775148, 'loss': 0.0050642635049046696, 'fairness': 10.458190175721093}
[3] Eval metrics for task 3 >> {'accuracy': 99.15252758756053, 'loss': 0.00015256537649876032, 'fairness': 0.1426265974615093}
---------------------------- Task 4 -----------------------
[4] 

In [6]:
metric_manager_callback.meters['fairness'].get_eer()

[0.0,
 16.261474225268337,
 17.819397588728183,
 18.874876258925788,
 16.154054741603943]

In [7]:
metric_manager_callback.meters['accuracy'].get_data()

array([[97.224,  0.   ,  0.   ,  0.   ,  0.   ],
       [65.152, 97.675,  0.   ,  0.   ,  0.   ],
       [46.112, 72.005, 99.153,  0.   ,  0.   ],
       [42.975, 67.749, 87.755, 98.468,  0.   ],
       [43.068, 57.003, 84.553, 70.863, 95.653]])

In [8]:
metric_manager_callback.meters['accuracy'].compute_overall()

79.10503304451898

In [9]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"fairness:{np.mean(metric_manager_callback.meters['fairness'].compute_overall())}")


accuracy:79.10503304451898
fairness:13.821960562905252
